In [1]:
import numpy as np
from sympy import * 
import sympy as sym
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import math
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib as mpl
import copy




#Statistik wie die Großen


#Arithmetischer Mittelwert

def mittelwert(Messreihe):
    return sum(Messreihe)/len(Messreihe)


#Empirische Varianz

def empVar(Messreihe):
    if len(Messreihe) < 2:
        return 'Die Messreihe muss mindestens zwei Messwerte beinhalten'
    empvar = 0
    mw = mittelwert(Messreihe)
    indexarray = np.arange(0,len(Messreihe))
    for index in indexarray:
        empvar += (Messreihe[index] - mw)**2
    return (1/(len(Messreihe)-1))*empvar


#Empirische Standardabweichung

def empStan(Messreihe):
    return empVar(Messreihe)**(1/2)


#Standardabweichung des Mittelwerts

def Stanmi(Messreihe):
    return empStan(Messreihe)/(len(Messreihe)**(1/2))


#Standardabweichung der empirischen Standardabweichung:

def StanempStan(Messreihe):
    return empStan(Messreihe)/((2*(len(Messreihe)-1))**(1/2))



#Gauß-Fehlerfortpflanzung
# Definiere die Symbole, Parameter, die Funktion, die Unsicherheiten und die Bestwerte:

#x, y = symbols('? ? ')
#params = [?,?]
#Bestwerte = [?,?]      #Bestwerte in gleicher Reihenfolge wie Symbole eingeben
#Unsicherheiten = [?,?]  #Unsicherheiten in gleicher Reihenfolge wie Symbole eingeben
#exp = ? 

def fehlerfort(exp, params, Bestwerte, Unsicherheiten):
    delexp = 0
    for i in np.arange(0, len(params)):
        delexp += (sym.lambdify(params, sym.diff(exp,params[i]), modules=['math'])(*Bestwerte)*Unsicherheiten[i])**2
    return delexp**(1/2)

# Korrelierte Fehlerfortpflanzung

def korfehlerfort(exp, params, Bestwerte, Unsicherheiten, Eingang):
    #x, y = symbols('? ? ')
    #Eingang= [?,?,?]  #Liste von allen Arrays mit allen Werten der Parameter um Korrelationskoeffizienten zu bestimmen
    #params = [?,?]
    #Bestwerte = [?,?]      #Bestwerte in gleicher Reihenfolge wie Symbole eingeben
    #Unsicherheiten = [?,?]  #Unsicherheiten in gleicher Reihenfolge wie Symbole eingeben
    #exp = ? 
    A = []
    for i in np.arange(len(params)):
        A.append(sym.lambdify(params, sym.diff(exp,params[i]), modules=['math'])(*Bestwerte))
    return np.sqrt(abs((np.dot(A,np.dot(kovmatrix(Eingang,Unsicherheiten),A)))))
        
        

#x, y = symbols('? ? ')
# Eingang= [?,?,?]  #Liste von allen Arrays mit allen Werten der Parameter um Korrelationskoeffizienten zu bestimmen
#params = [?,?]
#Bestwerte = [?,?]      #Bestwerte in gleicher Reihenfolge wie Symbole eingeben
#Unsicherheiten = [?,?]  #Unsicherheiten in gleicher Reihenfolge wie Symbole eingeben
#exp = ? 

# Kovarianzmatrix V:
def kovmatrix(Eingang, Unsicherheiten):
    if len(Eingang)!= len(Unsicherheiten):
        return 'Eingangsvariablen und Fehler stimmen nicht überein'
    V = []
    for i in np.arange(len(Eingang)):
        for n in np.arange(len(Eingang)):
            if i == n:
                V.append(Unsicherheiten[i]**2)
            else:
                V.append(empKor(Eingang[i], Eingang[n]))
    return np.reshape(V, (len(Eingang), len(Eingang)))
                
        
#Empirische Korrelation

def empKor(x,y):
    if len(x)!=len(y):
        return 'Datensätze müssen gleich lang sein'
    else:
        a = []
        for i in np.arange(len(x)):
            a.append((x[i]-mittelwert(x))*(y[i]-mittelwert(y)))
        return (1/(len(x)-1))*sum(a)
    

#Korrelationskoeffzienten

def korkoef(x,y):
    return empKor(x,y)/(empStan(x)*empStan(y))


# Einfacher t-test

def ttest(x, y, delta_x):
    return abs(x-y)/delta_x
    
    
# t-test mit zwei Ergebnissen

def ttest2(x, y, delta_x , delta_y):
    return abs(x-y)/(np.sqrt(delta_x**2+delta_y**2))

def gauß(x, A, sigma, x0, C):
    return A*np.exp( -(x-x0)**2/(2*sigma**2 )) + C 

In [2]:
from matplotlib.collections import PolyCollection
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import poisson

In [3]:
rtime = np.array(pd.read_csv('konstante_frequenz/r_calibration_HM1508.csv'))[:,0]
rsignal = np.array(pd.read_csv('konstante_frequenz/r_calibration_HM1508.csv'))[:,1]

plt.scatter(rtime[i], rsignal[i], color='blue', marker='.')

FileNotFoundError: [Errno 2] No such file or directory: 'konstante_frequenz/r_calibration_HM1508.csv'

In [ ]:
time = np.array([])
signal = np.array([])
Popt = []

index = np.arange(11)

for i in index:
    #print(type(i))
    time = np.append(time, np.array(pd.read_csv('konst_freq_2/'+str(i)+'_HM1508.csv'))[:,0])
    signal = np.append(signal, np.array(pd.read_csv('konst_freq_2/'+str(i)+'_HM1508.csv'))[:,1])
    time = time.reshape(i+1,2000)
    signal = signal.reshape(i+1,2000)
    #print(time.shape)
    #print(signal.shape)
    signal[i] = signal[i]*1/signal[i].max()
    if i == 0:
        #print('hell')
        referenz = copy.deepcopy(signal[0])
    #print(referenz)
    popt, pcov = curve_fit(gauß, time[i], signal[i], maxfev=100000, p0=[1, 0.0004, 0.001,  0.05])
    x = np.linspace(min(time[i]), max(time[i]), 10000)
    #plt.scatter(Time[i], Signal[i], marker='.')
    plt.scatter(time[i], signal[i], color='blue', marker='.')
    plt.scatter(time[i], signal[i]- gauß(time[i], *popt), color='red', marker='.')
    signal[i] = np.array(signal[i] - referenz)
    #Signal = np.array(Signal)
    #print(signal[i].shape)
    #plt.plot(x, gauß(x, *popt), color='r')
    plt.scatter(time[i], signal[i], color='green', marker='.')
    plt.grid()
    plt.show()
    
'''for i in index:
    time = np.append(time.flatten(), np.array(pd.read_csv('konstante_frequenz/'+str(i)+'_HM1508.csv'))[:,0], axis=0)
    signal = np.append(signal.flatten(), np.array(pd.read_csv('konstante_frequenz/'+str(i)+'_HM1508.csv'))[:,1], axis=0)
    time = time.reshape(i+1,2000)
    signal = signal.reshape(i+1,2000)
    print(time.shape)
    print(signal.shape)
    popt, pcov = curve_fit(gauß, time[i], signal[i], maxfev=100000, p0=[1, 0.0004, 0.001,  0.05])
    x = np.linspace(min(time[i]), max(time[i]), 10000)
    #plt.scatter(Time[i], Signal[i], marker='.')
    #Signal[i] = np.array(Signal[i] - gauß(Time[i], *popt))
    #Signal = np.array(Signal)
    print(signal[i].shape)
    #plt.plot(x, gauß(x, *popt), color='r')
    plt.scatter(time[i], signal[i], color='green', marker='.')
    plt.grid()
    plt.show()'''


In [ ]:
array = np.arange(0,2000,2)
ttime = np.delete(time,array,1)
tsignal = np.delete(signal,array,1)

In [ ]:
array = np.arange(0,2000,2)
tttime = np.delete(time,array,1)
ttsignal = np.delete(signal,array,1)

In [ ]:
ttime.shape

In [ ]:
for i in index:
    #popt, pcov = curve_fit(gauß, ttime[i], tsignal[i], maxfev=100000, p0=[1, 0.0004, 0.001,  0.05])
    #popt2, pcov2 = curve_fit(gauß, Time[i], Signal[i], maxfev=100000, p0=[1, 0.0004, 0.001,  0.05])
    x = np.linspace(min(ttime[i]), max(ttime[i]), 10000)
    #plt.plot(x, gauß(x, *popt), color='r')
    plt.scatter(time[i], signal[i], color='blue', marker='.')
    plt.scatter(tttime[i], ttsignal[i], color='red', marker='.')
    plt.scatter(ttime[i], tsignal[i], color='green', marker='.')
    
    #plt.xlim(0.0006,0.0014)
    plt.grid()
    plt.show()

In [ ]:
for i in index:
    print(tsignal[i][480:550].argmax())
    plt.scatter(ttime[i][480:550], tsignal[i][480:550], color='green', marker='.')
    plt.ylim(-0.0075,0.015)
    #plt.xlim(0.0006,0.0014)
    plt.grid()
    plt.show()

In [ ]:
for i in index:
    print(ttsignal[i][480:550].argmax())
    plt.scatter(tttime[i][400:550], ttsignal[i][400:550], color='green', marker='.')
    #plt.ylim(-0.0075,0.015)
    #plt.xlim(0.0006,0.0014)
    plt.grid()
    plt.show()

In [ ]:
tupils = []
for i in range(11):
    y = []
    for j in range(ttime[i].size):
        y.append((ttime[i][j],tsignal[i][j]))
    tupils.append(y)

In [ ]:
print(np.array(tupils).shape)

In [ ]:
np.random.seed(19680801)


def polygon_under_graph(x, y):
    """
    Construct the vertex list which defines the polygon filling the space under
    the (x, y) line graph. This assumes x is in ascending order.
    """
    return [(x[0], 0.), *zip(x, y), (x[-1], 0.)]


ax = plt.figure(figsize=(14,14)).add_subplot(projection='3d')

x = np.linspace(0., 10., 31)
#print(x)
lambdas = range(1, 12)
print(lambdas)

# verts[i] is a list of (x, y) pairs defining polygon i.
verts = [polygon_under_graph(x, poisson.pmf(l, x)) for l in lambdas]
#print(verts)
#facecolors = mpl.pyplot.colormaps['viridis_r'](np.linspace(0, 1, len(verts)))

poly = PolyCollection(tupils, alpha=.7)
ax.add_collection3d(poly, zs=lambdas, zdir='y')

ax.set(xlim=(0.0005, 0.0015), ylim=(1, 12), zlim=(0, 0.2),
       xlabel='x', ylabel=r'$\lambda$', zlabel='probability')

plt.show()

In [ ]:
facecolors = plt.colormaps()[2]
facecolors